In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd

class JobSpider(scrapy.Spider):
    name='job_spider'
        
    def start_requests(self):
        locations=input('Enter locations to search jobs seperating commas')
        jobs=input('Enter Jobs to search seperating commas')
        locations='Sri Lanka'
        jobs='Data Analysis'
        urls=[]
        try:
            if locations=='' and jobs=='':
                raise ValueError('Both Location and Job parameters are blank. Please try again.')
            elif locations=='':
                raise ValueError('Location parameter is blank. Please try again.')
            elif jobs=='':
                raise ValueError('Job parameters is blank. Please try again.')
        #except ValueError:
            #print('Error: ')
        finally:
            locations_list=locations.split(',')
            jobs_list=jobs.split(',')
            for loc in locations_list:
                for job in jobs_list:
                    urls.append('https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=' + job.replace(' ','+') + '&txtLocation=' + loc.replace(' ','+'))
            for url in urls:
                yield scrapy.Request(url=url,callback=self.parse_jobs_url)
    
    def parse_jobs_url(self,response):
        job_list=response.css('ul.new-joblist')
        job_urls=job_list.css('h2 ::attr(href)').extract()
        for job_url in job_urls:
            yield response.follow(url=job_url,callback=self.parse_job_data)

    def parse_job_data(self,response):
        job_title_uncleaned=response.css('h1::text').extract()
        job_title.append([t.strip() for t in job_title_uncleaned][0])
        company_uncleaned=response.css('h2::text').extract()
        company.append([t.strip() for t in company_uncleaned][0])
        key_skills_uncleaned=response.css('span.jd-skill-tag ::text').extract()
        key_skills_cleaned=[t.strip() for t in key_skills_uncleaned]
        key_skills_no_blank=[t for t in key_skills_cleaned if t!='']
        key_skills_string=', '.join(str(t) for t in key_skills_no_blank)
        key_skills.append(key_skills_string)
        print(key_skills_string)


def Save_to_CSV():
    new_job=pd.DataFrame(
                {'Title': job_title,
                'Company':company,
                'Skills':key_skills
                })
    new_job.to_csv('job_list.csv',index=False)


job_title=[]
company=[]
key_skills=[]

job_data=pd.DataFrame()
process=CrawlerProcess()
process.crawl(JobSpider)
process.start()
Save_to_CSV()


2020-08-16 21:54:32 [scrapy.utils.log] INFO: Scrapy 2.2.1 started (bot: scrapybot)
2020-08-16 21:54:32 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (tags/v3.8.5:580fbb0, Jul 20 2020, 15:57:54) [MSC v.1924 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 3.0, Platform Windows-10-10.0.19041-SP0
2020-08-16 21:54:32 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-08-16 21:54:32 [scrapy.crawler] INFO: Overridden settings:
{}
2020-08-16 21:54:32 [scrapy.extensions.telnet] INFO: Telnet Password: 7a769f1c108276dc
2020-08-16 21:54:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-08-16 21:54:32 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddlew

ReactorNotRestartable: 